In [1]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import os
from os.path import join
import sys
import scipy.stats
import numpy
import math
import pickle
import copy
import time
import random

import warnings

# Using Cobrapy 0.13.0
import cobra
import cobra.test
from cobra.flux_analysis.sampling import OptGPSampler
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types
#from cobra.flux_analysis.sampling import OptGPSampler

# Using Gurobi solver instead of GLPK
import gurobipy
from optlang import gurobi_interface

# Estabish handler for logger
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

# Verbose exception printing
%xmode

Exception reporting mode: Verbose


In [9]:
# Define functions

# Identify potentially gapfilled reactions
def findGapfilledRxn(model, exclude=[]):
    gapfilled = []
    transport = findTransports(model)
    if not type(exclude) is list:
        exclude = [exclude]
        
    for index in model.reactions:
        if len(list(index.genes)) == 0:
            if not index in model.boundary:
                if not index.id in exclude or not index.id in transport:
                    gapfilled.append(index.id)
    
    if len(gapfilled) > 0:
        print(str(len(gapfilled)) + ' metabolic reactions not associated with genes')
    
    return gapfilled

# Check for missing transport and exchange reactions
def missingRxns(model, extracellular=['e','Extracellular']):

    transporters = set(findTransports(model))
    exchanges = set([x.id for x in model.exchanges])
    
    missing_exchanges = []
    missing_transports = []
    
    for metabolite in model.metabolites:
        if not metabolite.compartment in extracellular or metabolite.id.split('_')[1] != 'e':
            continue

        curr_rxns = set([x.id for x in list(metabolite.reactions)])
        
        if bool(curr_rxns & transporters) == False:
            missing_transports.append(metabolite.id)
        if bool(curr_rxns & exchanges) == False:
            missing_exchanges.append(metabolite.id)
    
    if len(missing_transports) != 0:
        print(str(len(missing_transports)) + ' extracellular metabolites are missing transport reactions')
    if len(missing_exchanges) != 0:
        print(str(len(missing_exchanges)) + ' extracellular metabolites are missing exchange reactions')
        
    return missing_transports, missing_exchanges


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(raw_model, cytosol='Cytosol'):
    
    with raw_model as model:
        
        # Close all exchanges
        for index in model.boundary:
            model.reactions.get_by_id(index.id).lower_bound = 0.
        
        # Identify all metabolites that are produced within the network
        demand_metabolites = [x.reactants[0].id for x in model.demands if len(x.reactants) > 0] + [x.products[0].id for x in model.demands if len(x.products) > 0]

        free = []
        for index in model.metabolites: 
            if index.id in demand_metabolites:
                continue
            elif not index.compartment in cytosol:
                continue
            else:
                demand = model.add_boundary(index, type='demand')
                model.objective = demand
                obj_val = model.slim_optimize(error_value=0.)
                if obj_val > 1e-8:
                    free.append(index.id)
                model.remove_reactions([demand])
    
    if len(free) > 0:
        print(str(len(free)) + ' metabolites are generated for free')

    return(free)


# Check for mass and charge balance in reactions
def checkBalance(raw_model, exclude=[]):
    
    with raw_model as model:
        imbalanced = []
        mass_imbal = 0
        charge_imbal = 0
        elem_set = set()
        for metabolite in model.metabolites:
            try:
                elem_set |= set(metabolite.elements.keys())
            except:
                pass
        
        if len(elem_set) == 0:
            imbalanced = model.reactions
            mass_imbal = len(model.reactions)
            charge_imbal = len(model.reactions)
            print('No elemental data associated with metabolites!')
        
        else:
            if not type(exclude) is list: 
                exclude = [exclude]
            for index in model.reactions:
                if index in model.boundary or index.id in exclude:
                    continue

                else:
                    try:
                        test = index.check_mass_balance()
                    except ValueError:
                        continue

                    if len(list(test)) > 0:
                        imbalanced.append(index.id)

                        if 'charge' in test.keys():
                            charge_imbal += 1
                        if len(set(test.keys()).intersection(elem_set)) > 0:
                            mass_imbal += 1

    if mass_imbal != 0:
        print(str(mass_imbal) + ' reactions are mass imbalanced')
    if charge_imbal != 0:
        print(str(charge_imbal) + ' reactions are charge imbalanced')
    
    return(imbalanced)


# Identifies blocked reactions, 1% cutoff for fraction of optimum
def blockedReactions(model):
    
    blocked = cobra.flux_analysis.variability.find_blocked_reactions(model)
    
    if len(blocked) != 0:
        print(str(len(blocked)) + ' reactions are blocked')
        
    return blocked


# Checks the quality of models by a couple metrics and returns problems
def checkQuality(model, exclude=[], cytosol='c'):
    
    start_time = time.time()
    
    if model.name != None:
        model_name = model.name
    else:
        model_name = 'model'
    
    gaps = findGapfilledRxn(model, exclude)
    freemass = checkFreeMass(model, cytosol)
    balance = checkBalance(model, exclude)
    #blocked = blockedReactions(model)
    trans, exch = missingRxns(model)
    
    test = gaps + freemass + balance
    if len(test) == 0:
        print('No inconsistencies detected')
    
    duration = int(round(time.time() - start_time))
    print('Took ' + str(duration) + ' seconds to analyze ' + model_name) 

    return gaps, freemass, balance, trans, exch


# Trace back through reactions immediately adjacent to a given reaction to identify blocked precursor synthesis
def checkPrecursors(model, reaction):
    
    if isinstance(reaction, str) == True:
        reaction = model.reactions.get_by_id(reaction)
    
    model.objective = reaction
    obj_val = max(model.optimize(objective_sense='maximize').objective_value, abs(model.optimize(objective_sense='minimize').objective_value))

    if obj_val > 0.001:
        print('Able to produce all precursors for this reaction.')
        return None
    
    else:
        reactants = reaction.reactants
        check = 0
        
        for reactant in reactants:
            sub_reactions = list(reactant.reactions)
            
            for sub_reaction in sub_reactions:
                model.objective = sub_reaction
                obj_val = max(model.optimize(objective_sense='maximize').objective_value, abs(model.optimize(objective_sense='minimize').objective_value))

                if obj_val < 0.001 and reactant in sub_reaction.products:                    
                    print('Cannot acquire ' + str(reactant.id) + ' via ' + str(sub_reaction.id))
                elif obj_val < 0.001 and check < 1 and reactant in sub_reaction.reactants:
                    print(str(reactant.id) + ' not produced in any reactions.')
                    check += 1

#------------------------------------------------------------------------------------#

# Function to calculate doubling time from objective value
def doubling(model):
    with model as m:
        growth = (1 / float(m.slim_optimize())) * 3600 
        growth = str(round(growth, 3)) + ' minutes'
    return growth


# Function to change media condition based on a list
def changeMedia(model, media_list):
    for index in model.exchanges:
        if index.id in media_list:
            model.reactions.get_by_id(index.id).lower_bound = -1000.0
        else:
            model.reactions.get_by_id(index.id).lower_bound = 0.0
            
    return model


# Rough transcriptomic integration
def roughContextualize(model, transcript_profile, condition):
    
    orig_OV = model.optimize().objective_value

    model_context = copy.deepcopy(model)
    abundances = []
    
    with open(transcript_profile, 'r') as transcription:
        transcript_dict = {}
        abundances = []
        header = transcription.readline().strip().split(',')
        column = header.index(condition)
        for line in transcription:
            line = line.split(',')
            transcript_dict[line[0]] = float(line[column])
            abundances.append(float(line[column]))
    
    min_transcription = numpy.percentile(abundances, 50)
    penalty_bound = 10
    
    hits = 0
    fails = 0
    for gene in list(model_context.genes):
    
        gene = gene.name
    
        try:
            curr_rxns = list(model_context.genes.get_by_id(gene).reactions)
            hits += 1
        except KeyError:
            fails += 1
            continue
    
        try:
            curr_transcription = transcript_dict[gene]
        except KeyError:
            continue

        for reaction in curr_rxns:
            curr_id = reaction.id
        
            if curr_transcription >= min_transcription:            
                model_context.reactions.get_by_id(curr_id).lower_bound = -1000
                model_context.reactions.get_by_id(curr_id).upper_bound = 1000
            elif curr_transcription < min_transcription:
                if model_context.reactions.get_by_id(curr_id).lower_bound != 0:
                    model_context.reactions.get_by_id(curr_id).lower_bound = -penalty_bound
                if model_context.reactions.get_by_id(curr_id).upper_bound != 0:
                    model_context.reactions.get_by_id(curr_id).upper_bound = penalty_bound
                  

    #print('Gene hits across data types: ' + str(hits))
    #print('KeyErrors across data types: ' + str(fails) + '\n')
    new_OV = model_context.optimize().objective_value
    print('New objective value: ' + str(new_OV))
    print('Contextualized doubling time: ' + doubling(new_OV))
    
    return(model_context)


# Checks for availability of reactants of a given reaction
def availability(model, target_rxn):

    precursors = model.reactions.get_by_id(target_rxn).reactants
    
    total = 0
    unsuccessful = set()
    limited = set()
    for precursor in precursors:
        precursor_rxn = list(model.metabolites.get_by_id(precursor.id).reactions)
        
        for rxn in precursor_rxn:
            if rxn.id == target_rxn:
                continue
            elif precursor in model.reactions.get_by_id(rxn.id).reactants:
                
                model.objective = rxn
                obj_val = model.slim_optimize()
                if obj_val < 1e-8:
                    unsuccessful |= set([rxn.id])
                    limited |= set([precursor.id])
    
    print('Failed reactions: ' + str(len(unsuccessful)))
    print('Limiting reactants: ' + str(len(limited)))
    
    return unsuccessful, limited

# Removes all metabolites in a list of metabolite ids and all reactions associated with them
def removeAll(model, metabolite_list):
    
    new_model = copy.deepcopy(model)
    
    for metabolite in metabolite_list:
        try:
            metabolite = new_model.metabolites.get_by_id(metabolite)
            new_model.remove_reactions(metabolite.reactions)
            new_model.remove_metabolites([metabolite])
        except KeyError:
            print(metabolite + ' not found')
            continue
        
    return new_model


# Identify transport reactions (for any number compartments)
def findTransports(model):
    transporters = []
    compartments = set(list(model.compartments))
    if len(compartments) == 1:
        raise Exception('Model only has one compartment!')
    
    for reaction in model.reactions:
        
        reactant_compartments = set([x.compartment for x in reaction.reactants])
        product_compartments = set([x.compartment for x in reaction.products])
        reactant_baseID = set([x.id.split('_')[0] for x in reaction.reactants])
        product_baseID = set([x.id.split('_')[0] for x in reaction.products])
        
        if reactant_compartments == product_compartments and reactant_baseID != product_baseID:
            continue
        elif bool(compartments & reactant_compartments) == True and bool(compartments & product_compartments) == True:
            transporters.append(reaction.id)
        
    return(transporters)

# Removes a given percentage of reactions from a model, ignoring objective
def generate_gaps(model, percentage=0.2, prune=False, ignore=[]):
    number_to_remove = int(round(len(model.reactions) * percentage))
    rxn_ids = [x.id for x in model.reactions]
    random.shuffle(rxn_ids)
    rxns_to_remove = rxn_ids[-number_to_remove:]
        
    for rxn in ignore:
        try:
            rxns_to_remove.remove(rxn)
        except ValueError:
            continue
    
    truncated_model = copy.deepcopy(model)
    truncated_model.remove_reactions(rxns_to_remove)
    
    if prune == True:
        unused_cpds = prune_unused_metabolites(truncated_model)
    
    print('Reactions removed: ' + str(len(rxns_to_remove)))
    print('New objective value: ' + str(truncated_model.slim_optimize()))
    return truncated_model, rxns_to_remove


# Calculates the sum of fluxes for a given model
def sum_of_fluxes(model):
    
    with model as m:
        solution = m.optimize()
        flux_sum = sum(list(solution.fluxes))
        
    return flux_sum


In [15]:
# Very fast and efficient gap filling function
def pFBA_GapFill(model, bag, obj=None, obj_lb=10., obj_constraint=False,
                 iters=1, tasks=None, task_lb=0.05, 
                 add_exchanges=True, extracellular='e'):
    '''
    Function that utilizes iterations of pFBA solution with a universal reaction bag 
    in order to gapfill a model.
    
    Parameters
    ----------
    model : cobra.Model
        Model to be gapfilled
    bag : cobra.Model
        Reaction bag reference to use during gapfilling
    obj : string
        Reaction ID for objective function in model to be gapfilled.
    obj_lb : float
        Lower bound for objective function
    obj_constraint : bool
        Sets objective as contstraint which must be maximized
    tasks : list or None
        List of reactions IDs (strings) of metabolic tasks 
        to set a minimum lower bound for
    task_lb : float
        Lower bound for any metabolic tasks
    iters : int
        Number of gapfilling rounds. Unique reactions from each round are 
        saved and the union is added simulatneously to the model
    add_exchanges : bool
        Identifies extracellular metabolites added during gapfilling that
        are not associated with exchange reactions and creates them
    extracellular : string
        Label for extracellular compartment of model
    '''
    start_time = time.time()
    
    # Save some basic network info for downstream membership testing
    orig_rxn_ids = set([str(x.id) for x in model.reactions])
    orig_cpd_ids = set([str(y.id) for y in model.metabolites])
    univ_rxn_ids = set([str(z.id) for z in bag.reactions])
    
    # Find overlap in model and reaction bag
    overlap_rxn_ids = univ_rxn_ids.intersection(orig_rxn_ids)
    
    # Get model objective reaction ID
    if obj == None:
        obj = get_objective(model)
    else:
        obj = obj
    
    # Modify universal reaction bag
    new_rxn_ids = set()
    print('Creating universal model...')
    with bag as universal:

        # Remove overlapping reactions from universal bag, and reset objective if needed
        for rxn in overlap_rxn_ids: 
            universal.reactions.get_by_id(rxn).remove_from_model()
        
        # Set objective in universal if told by user
        # Made constraint as fraction of minimum in next step
        if obj_constraint == True:
            universal.add_reactions([model.reactions.get_by_id(obj)])
            universal.objective = obj
            orig_rxn_ids.remove(obj)
            orig_rxns = []
            for rxn in orig_rxn_ids: 
                orig_rxns.append(copy.deepcopy(model.reactions.get_by_id(rxn)))
        else:
            orig_rxns = list(copy.deepcopy(model.reactions))
            
        # Add pFBA to universal model and add model reactions
        add_pfba(universal)
        universal = copy.deepcopy(universal) # reset solver
        universal.add_reactions(orig_rxns)
        
        # If previous objective not set as constraint, set minimum lower bound
        if obj_constraint == False: 
            universal.reactions.get_by_id(obj).lower_bound = obj_lb
    
        # Set metabolic tasks that must carry flux in gapfilled solution
        if tasks != None:
            for task in tasks:                    
                universal.reactions.get_by_id(task).lower_bound = task_lb
                
        # Run FBA and save solution
        print('Optimizing model with combined reactions...')
        solution = universal.optimize()

        if iters > 1:
            print('Generating flux sampling object...')
            optgp_object = OptGPSampler(universal, processes=4)
        
            # Assess the sampled flux distributions
            print('Sampling ' + str(iters) + ' flux distributions...')
            flux_samples = optgp_object.sample(iters)
            rxns = list(flux_samples.columns)
            for distribution in flux_samples.iterrows():
                for flux in range(0, len(list(distribution[1]))):
                    if abs(list(distribution[1])[flux]) > 1e-6:
                        new_rxn_ids |= set([rxns[flux]]).difference(orig_rxn_ids)
        else:
            rxns = list(solution.fluxes.index)
            fluxes = list(solution.fluxes)
            for flux in range(0, len(fluxes)):
                if abs(fluxes[flux]) > 1e-6:
                    new_rxn_ids |= set([rxns[flux]])
    
    # Screen new reaction IDs
    if obj in new_rxn_ids: new_rxn_ids.remove(obj)
    for rxn in orig_rxn_ids:
        try:
            new_rxn_ids.remove(rxn)
        except:
            continue
    
    # Get reactions and metabolites to be added to the model
    print('Retrieving reactions and metabolites needed for gapfilling...')
    new_rxns = copy.deepcopy([bag.reactions.get_by_id(rxn) for rxn in new_rxn_ids])
    new_cpd_ids = set()
    for rxn in new_rxns: new_cpd_ids |= set([str(x.id) for x in list(rxn.metabolites)])
    new_cpd_ids = new_cpd_ids.difference(orig_cpd_ids)
    new_cpds = copy.deepcopy([bag.metabolites.get_by_id(cpd) for cpd in new_cpd_ids])
    
    # Copy model and gapfill 
    print('Gapfilling model...')
    new_model = copy.deepcopy(model)
    new_model.add_metabolites(new_cpds)
    new_model.add_reactions(new_rxns)
    
    # Identify extracellular metabolites with no exchanges
    if add_exchanges == True:
        new_exchanges = extend_exchanges(new_model, new_cpd_ids, extracellular)
        if len(new_exchanges) > 0: new_rxn_ids |= new_exchanges
    
    duration = int(round(time.time() - start_time))
    print('Took ' + str(duration) + ' seconds to gapfill ' + str(len(new_rxn_ids)) + \
          ' reactions and ' + str(len(new_cpd_ids)) + ' metabolites.') 
    
    new_obj_val = new_model.slim_optimize()
    if new_obj_val > 1e-6:
        print('Gapfilled model objective now carries flux (' + str(new_obj_val) + ').')
    else:
        print('Gapfilled model objective still does not carry flux.')
    
    return new_model


# Adds missing exchanges for extracellulart metbaolites
def extend_exchanges(model, cpd_ids, ex):
    
    model_exchanges = set(find_boundary_types(model, 'exchange', external_compartment=ex))
    new_ex_ids = set()
    
    for cpd in cpd_ids:
        cpd = model.metabolites.get_by_id(cpd)
        if str(cpd.compartment) != ex:
            continue
        else:
            if bool(set(cpd.reactions) & model_exchanges) == False:
                try:
                    new_id = 'EX_' + cpd.id
                    model.add_boundary(cpd, type='exchange', reaction_id=new_id, lb=-1000.0, ub=1000.0)
                    new_ex_ids |= set([new_id])
                except ValueError:
                    pass

    return new_ex_ids


# Returns the reaction ID of the objective reaction
def get_objective(model):
    
    if len(list(model.objective.variables)) == 0:
        raise IndexError('Model has no objective set.')
    
    expression = str(model.objective.expression).split()
    if 'reverse' in expression[0]:
        obj_id = expression[2].split('*')[-1]
    else:
        obj_id = expression[0].split('*')[-1]
            
    return obj_id


### Toy Model

In [100]:
toy = cobra.io.read_sbml_model('data/toy_model.sbml')

In [101]:
toy

Name,toy_model
Memory address,0x07f2274dbb590
Number of metabolites,7
Number of reactions,9
Objective expression,0
Compartments,"c, e"


In [102]:
# Remove specific reactions
test_model = copy.deepcopy(toy)
test_model.remove_reactions(['rxn3','rxn4','rxn5','rxn6'])

In [103]:
test_model.objective = 'biomass_rxn'

In [107]:
test_gapfill = pFBA_GapFill(test_model, toy, iters=2, extracellular='e')

Creating universal model...
Optimizing model with new reactions...
Generating flux sampling object...
Sampling 2 flux distributions...
Retrieving reactions and metabolites needed for gapfilling...
Gapfilling model...
Took 0 seconds to gapfill 4 reactions and 0 metabolites.
Gapfilled model objective now carries flux.


In [105]:
test_gapfill

Name,toy_model
Memory address,0x07f22d3afbb90
Number of metabolites,7
Number of reactions,9
Objective expression,1.0*biomass_rxn - 1.0*biomass_rxn_reverse_1a99c
Compartments,"c, e"


In [159]:
test_gapfill.slim_optimize()

1000.0

### iML1515

In [112]:
iML1515 = cobra.io.read_sbml_model('data/iML1515.xml')

In [115]:
iML1515

Name,iML1515
Memory address,0x07f2281e6f0d0
Number of metabolites,1877
Number of reactions,2712
Objective expression,-1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685 + 1.0*BIOMASS_Ec_iML1515_core_75p37M
Compartments,"periplasm, cytosol, extracellular space"


In [114]:
test_model, removed_rxns = generate_gaps(iML1515, percentage=0.1, prune=True, ignore=['BIOMASS_Ec_iML1515_core_75p37M_reverse','BIOMASS_Ec_iML1515_core_75p37M'])

Reactions removed: 271
New objective value: nan


In [116]:
test_model

Name,iML1515
Memory address,0x07f227709f590
Number of metabolites,1853
Number of reactions,2441
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"periplasm, cytosol, extracellular space"


In [117]:
test_gapfill = pFBA_GapFill(test_model, iML1515, obj_lb=10.)

Creating universal model...


ContainerAlreadyContains: Container '<optlang.container.Container object at 0x7f22901fa290>' already contains an object with name 'fixed_objective_6c94da15-a6f0-11e8-bbc5-f8633f05fb68'.

In [73]:
test_gapfill

Name,iML1515
Memory address,0x07f2276358950
Number of metabolites,1877
Number of reactions,2441
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"periplasm, cytosol, extracellular space"


### New C. difficile model

In [16]:
# Read in models
universal = cobra.io.load_json_model('data/universal.json')
cd630_PATRIC = cobra.io.read_sbml_model('data/PATRIC/temp_cd630PATRIC.sbml') # partially curated

In [17]:
test_gapfill = pFBA_GapFill(cd630_PATRIC, universal, iters=1, obj_lb=50., extracellular='Extracellular')

Creating universal model...
Optimizing model with new reactions...
Retrieving reactions and metabolites needed for gapfilling...
Gapfilling model...
Took 84 seconds to gapfill 23 reactions and 1 metabolites.
Gapfilled model objective now carries flux (97.8947890902).


In [18]:
doubling(test_gapfill)

'36.774 minutes'

In [22]:
test_gapfill

Name,272563.8
Memory address,0x07f1826246110
Number of metabolites,1248
Number of reactions,1193
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"Cytosol, Extracellular"


In [20]:
unused = prune_unused_metabolites(test_gapfill)

In [23]:
new_rxns = set([str(x.id) for x in test_gapfill.reactions]).difference(set([str(y.id) for y in cd630_PATRIC.reactions]))

In [24]:
new_rxns

{'rxn00553_c',
 'rxn00620_c',
 'rxn01352_c',
 'rxn01520_c',
 'rxn02011_c',
 'rxn02286_c',
 'rxn03852_c',
 'rxn05291_c',
 'rxn05293_c',
 'rxn07466_c',
 'rxn08040_c',
 'rxn09016_c',
 'rxn09063_c',
 'rxn09065_c',
 'rxn09119_c',
 'rxn09124_c',
 'rxn09147_c',
 'rxn09163_c',
 'rxn09341_c',
 'rxn09429_c',
 'rxn09433_c',
 'rxn13012_c',
 'rxn13914_c'}

In [45]:
test_gapfill.reactions.get_by_id('rxn00553_c')

Reaction identifier,rxn00553_c
Name,UDP-glucose:D-fructose-6-phosphate 2-alpha-D-glucosyltransferase
Memory address,0x07f1825e94350
Stoichiometry,cpd00026_c + cpd00072_c <=> cpd00014_c + cpd01693_c UDP-glucose + D-fructose-6-phosphate <=> UDP + 6-Phosphosucrose
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [48]:
cobra.io.write_sbml_model(test_gapfill, 'data/cd630_gapfilled.sbml')

In [23]:
new_rxns = set([str(x.id) for x in test_gapfill.reactions]).difference(set([str(y.id) for y in cd630_PATRIC.reactions]))

In [24]:
new_rxns

{'rxn00553_c',
 'rxn00620_c',
 'rxn02011_c',
 'rxn02286_c',
 'rxn03852_c',
 'rxn05291_c',
 'rxn05293_c',
 'rxn07466_c',
 'rxn08040_c',
 'rxn09063_c',
 'rxn09065_c',
 'rxn09145_c',
 'rxn09147_c',
 'rxn09163_c',
 'rxn09429_c',
 'rxn09433_c',
 'rxn13782_c',
 'rxn13783_c',
 'rxn13784_c',
 'rxn13914_c'}

In [ ]:


#need to probably fix reversibility of gapfilled reactions...



In [44]:
test_gapfill.reactions.get_by_id('rxn13784_c')

Reaction identifier,rxn13784_c
Name,RNA transcription
Memory address,0x07efcf2ee2c50
Stoichiometry,--> cpd17043_c --> RNA transcription
GPR,
Lower bound,0.0
Upper bound,1000.0


In [47]:
no_genes, freemass, unbalanced, no_transport, no_exchange = checkQuality(test_gapfill)

67 metabolic reactions not associated with genes
26 reactions are mass imbalanced
55 reactions are charge imbalanced
1 extracellular metabolites are missing transport reactions
Took 2 seconds to analyze Clostridium difficile 630


In [30]:
gaps

['EX_cpd11574_e',
 'EX_cpd10516_e',
 'EX_cpd01188_e',
 'EX_cpd00028_e',
 'EX_cpd00010_e',
 'EX_cpd00048_e',
 'EX_cpd11606_e',
 'rxn09030_c',
 'turanose_isomerization_c',
 'rxn18579_c',
 'rxn05165_2_c',
 'rxn32054_c',
 'rxn00804_c',
 'ID001_c',
 'ID002_c',
 'ID003_c',
 'ID004_c',
 'ID005_c',
 'rxn11336_c',
 'rxn12566_c',
 'ID006_c',
 'ID007_c',
 'ID008_c',
 'rxn09271_c',
 'ID009_c',
 'rxn08061_c',
 'rxn05602_c',
 'rxn10171_c',
 'rxn09172_c',
 'dna_rxn',
 'rna_rxn',
 'protein_rxn',
 'cellwall_rxn',
 'lipid_rxn',
 'cofactor_rxn',
 'biomass',
 'rxn08686_c',
 'rxn09031_c',
 'rxn05467_c',
 'rxn08186_c',
 'rxn08952_c',
 'rxn10161_c',
 'rxn05175_c',
 'rxn08304_c',
 u'rxn09145_c',
 u'rxn00620_c',
 u'rxn02286_c',
 u'rxn09433_c',
 u'rxn00553_c',
 u'rxn09063_c',
 u'rxn05293_c',
 u'rxn09065_c',
 u'rxn05291_c',
 u'rxn13914_c',
 u'rxn02011_c',
 u'rxn03852_c',
 u'rxn08040_c',
 u'rxn09147_c',
 u'rxn07466_c',
 u'rxn09429_c',
 u'rxn09163_c']

In [5]:
test_gapfill = pFBA_GapFill(cd630_PATRIC, universal, iters=1000, obj_lb=50., extracellular='Extracellular')

Creating universal model...
Optimizing model with new reactions...
Retrieving reactions and metabolites needed for gapfilling...
Gapfilling model...
Took 84 seconds to gapfill 20 reactions and 0 metabolites.
Gapfilled model objective now carries flux (60.6159042045).
